In [1]:
!pip install html5lib
!pip install tqdm
!pip install pandas
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import html5lib

In [2]:
url = "https://books.toscrape.com"
response = requests.get(url)
soup = BeautifulSoup(response.text)

In [19]:
all_current = soup.find_all("li", {"class": "current"})
current = all_current[0]
encoded = current.encode_contents().decode("utf-8")
splits = encoded.split("\n")
page_string = splits[2].strip()
splits = page_string.split("of")
page_count_string = splits[1].strip()
page_count = int(page_count_string)

page_min = page_count - 9
page_max = page_count + 1

link_list = []
for i in range(page_min, page_max):
    page = f'https://books.toscrape.com/catalogue/page-{i}.html'
    
    for i in soup.findAll("h3"):
        for link in i.find_all('a',href = True):
            part_link = link["href"]                                  
            full_link = 'https://books.toscrape.com/' + part_link
            link_list.append(full_link)

print(len(link_list))

200
